<a href="https://colab.research.google.com/github/RaviKcse08/datascience_projs/blob/main/CC_FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_pickle('2018-05-14.pkl')
data.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
412660,412660,2018-05-14 00:00:01,851,9335,61.65,3715201,43,0,0
412661,412661,2018-05-14 00:01:46,2462,8563,125.74,3715306,43,0,0
412662,412662,2018-05-14 00:02:22,923,1514,97.75,3715342,43,0,0
412663,412663,2018-05-14 00:03:02,3142,3268,11.09,3715382,43,0,0
412664,412664,2018-05-14 00:03:44,3797,2609,111.90,3715424,43,0,0
